In [0]:
import pandas as pd

# load data and take sample
visit_data = pd.read_csv("air_visit_data.csv")

# filter out records without air_store_id
# integData2 = integData[pd.notnull(integData["air_store_id"])]

In [0]:
visit_data.head(5)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


variables to be created (create based on Datetime column) :
1. quarter, 
2. season (winter, summer), 
3. month, 
4. week, 
5. day of week
6. Days to the 25th of previous month (Japanese people usually get their salary at the 25th of each month)

1. quarter, 3. month, 4. week, 5. day of week

In [0]:
visit_data["visit_date"] = pd.to_datetime(visit_data["visit_date"])

# 1. quarter, 3. month, 4. week, 5. day of week
visit_data["quarter"] = visit_data["visit_date"].dt.quarter
visit_data["month"] = visit_data["visit_date"].dt.month
visit_data["week"] = visit_data["visit_date"].dt.week
visit_data["dayofweek"] = visit_data["visit_date"].dt.dayofweek # 0 (sunday) - 6

2. season
"""
Japan has four distinct seasons: 
March to May is spring; 
June to August is summer; 
September to November is autumn; 
and December to February is winter. 
Each season has very different temperatures and climates
"""

In [0]:
# 2. season
"""
Japan has four distinct seasons: 
March to May is spring; 
June to August is summer; 
September to November is autumn; 
and December to February is winter. 
Each season has very different temperatures and climates
"""
# integData2["month"].unique()

def seasonLabel(row):
    if row['month'] in [3,4,5]:
        return 'spring'
    if row['month'] in [6,7,8]:
        return 'summer'
    if row['month'] in [9,10,11]:
        return 'autumn'
    if row['month'] in [12,1,2]:
        return 'winter'
    
visit_data["season"] = visit_data.apply(lambda row:seasonLabel(row), axis=1)  
    

6. days to the 25th of previous month

In [0]:
# 6. days to the end of each month 
# logic: days in month - day of month
visit_data["dayofmonth"] = visit_data["visit_date"].dt.day    
# visit_data["daysinmonth"] = visit_data["visit_date"].dt.daysinmonth 
# visit_data["daysToMonthEnd"] = visit_data["daysinmonth"] - visit_data["dayofmonth"]  
visit_data["daysinPrevmonth"] = (visit_data["visit_date"] - pd.DateOffset(months=1)).dt.daysinmonth 


In [0]:
# 7. Days to the 25th of previous month (Japanese people usually get their salary at the 25th of each month)
# create days to the 25th of previous month
# 7.1 day of month >= 25, day of month - 25
# 7.2 day of month < 25, (days in month - 25) + day of month

def daysToPrev25th(row):
    TARGET_DATE = 25
    if row['dayofmonth'] >= 25:
        return row['dayofmonth'] - TARGET_DATE
    else:
        return row['daysinPrevmonth'] - TARGET_DATE + row['dayofmonth']

visit_data["daysToPrev25th"] = visit_data.apply(lambda row:daysToPrev25th(row), axis=1)

In [0]:
visit_data.head(5)

,air_store_id,visit_date,visitors,quarter,month,week,dayofweek,season,dayofmonth,daysinPrevmonth,daysToPrev25th
0,air_ba937bf13d40fb24,2016-01-13,25,1,1,2,2,winter,13,31,19
1,air_ba937bf13d40fb24,2016-01-14,32,1,1,2,3,winter,14,31,20
2,air_ba937bf13d40fb24,2016-01-15,29,1,1,2,4,winter,15,31,21
3,air_ba937bf13d40fb24,2016-01-16,22,1,1,2,5,winter,16,31,22
4,air_ba937bf13d40fb24,2016-01-18,6,1,1,3,0,winter,18,31,24
